In [429]:
### pip is the package installer for python3. The command "py -m pip install"
### installs the packages where in "-m pip" executes pip using the latest python interpreter installed in windows.

import os
import subprocess
import re

### packages for reading excel files
from openpyxl import Workbook
from openpyxl import load_workbook

### packages for writing word documents
from docx import Document
from docx.shared import Cm
import docx
from docx.enum.dml import MSO_THEME_COLOR_INDEX
#py -m pip install --user python-docx

### packages for webscraping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

#Setting up selenium driver
#Install the corresponding version of chromedriver on https://sites.google.com/a/chromium.org/chromedriver/downloads and place the file in a map. Store the path to the file in the PATH variable below.
'''
option = webdriver.ChromeOptions()
option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=option)
'''
# package to read the pdf files
from tabula import read_pdf
from tabulate import tabulate
import tabula

import csv

import re
import requests as rq
import json
import pprint

In [430]:
# Creating a function that allows for the creation of working hyperlinks

def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run ()
    r._r.append (hyperlink)

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    r.font.underline = True

    return hyperlink


In [431]:
###########Input
print("BRCA1/BRCA2?")
brca_input = input()
brca_input = brca_input.upper()
print(brca_input)
print("Insert nucleotide")
search = input()

if ".c" in search:
    search_c = search

BRCA1/BRCA2?
BRCA2
Insert nucleotide


In [432]:


####Setting up the path to the location of the excel files and pdf file

folder_location = "C:/Users/jensv/Desktop/BRCA_-_prostate_cancer_and_ovarian_cancer_-_PARPi"
functional_categorization = "/2020 BRCA1 functional categorization of BRCA1 VUS - CCR.xlsx"
variants_excel = "/allenigmavariants_BICsubmission_2013-07-01.xlsx"
bicbnl = "/bicbnl 27  fh 2-10-13_nov2013.xlsx"
enigma = "/BRCA_Multifac_published data_ENIGMAwebsite 2015-03-27.xlsx"
pdf_lindor = "/Lindor 2012.pdf"
cmg_brca1 = "/BRCA1 CMG 05 2021.txt"
cmg_brca2 = "/BRCA2 CMG 05 2021.txt"
brca1_exchange = "/brca1_exchange"
brca2_exchange = "/brca2_exchange"
new_path = "C:/Users/jensv/Stage_UZA"

In [433]:
#Word document

### Create word document
document = Document()
sections = document.sections
for section in sections:
    section.top_margin = Cm(1)
    section.bottom_margin = Cm(1)
    section.left_margin = Cm(1)
    section.right_margin = Cm(1)



In [434]:
### UZA CMG
### priority: 1
### The script will open the text file depending on if the gene input is BRCA1/2

document.add_heading("UZA CMG")

table = document.add_table(rows=1, cols=3)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'Nuc name'
hdr_cells[2].text = 'Mut effect'

if brca_input == "BRCA1":
    full_path = folder_location + cmg_brca1
    with open(full_path) as csv_file:
        # The script will open the text file using csv.reader but using the tab ("\t") delimiter so it will iterate over each line of text separated by tabs.
        csv_reader = csv.reader(csv_file, delimiter="\t")
        count = 0
        hit = False
        for row in csv_reader:
            for cell in row:
                if search in cell:
                    row_cells = table.add_row().cells
                    row_cells[0].text = brca_input
                    row_cells[1].text = cell
                    hit = True
                    count = count + 1
                elif hit == True:
                    count = count + 1
                    hit = False
                elif count == 2:
                    count = 0
                    row_cells[2].text = cell

elif brca_input == "BRCA2":
    full_path = folder_location + cmg_brca2
    with open(full_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter="\t")
        count = 0
        hit = False
        for row in csv_reader:
            for cell in row:
                if search in cell:
                    row_cells = table.add_row().cells
                    row_cells[0].text = brca_input
                    row_cells[1].text = cell
                    hit = True
                    count = count + 1
                elif hit == True:
                    count = count + 1
                    hit = False
                elif count == 2:
                    count = 0
                    row_cells[2].text = cell
                    



In [463]:
### Downloading the brca exchange databases locally
download_database = "YES"
if download_database == "YES":
    query_brca1 = """
https://brcaexchange.org/backend/data/
?format=json
&search_term=BRCA1
&include=Variant_in_ENIGMA
&include=Variant_in_ClinVar
&include=Variant_in_1000_Genomes
&include=Variant_in_ExAC
&include=Variant_in_LOVD
&include=Variant_in_BIC
&include=Variant_in_ESP
&include=Variant_in_exLOVD
&include=Variant_in_Findlay_BRCA1_Ring_Function_Scores
&include=Variant_in_GnomAD
&include=Variant_in_GnomADv3
"""
    response = rq.get(query_brca1.replace('\n', '')).json()['data']
    with open('brca1_exchange', 'w') as brca1_data:
        json.dump(response, brca1_data)

    query_brca2 = """
https://brcaexchange.org/backend/data/
?format=json
&search_term=BRCA1
&include=Variant_in_ENIGMA
&include=Variant_in_ClinVar
&include=Variant_in_1000_Genomes
&include=Variant_in_ExAC
&include=Variant_in_LOVD
&include=Variant_in_BIC
&include=Variant_in_ESP
&include=Variant_in_exLOVD
&include=Variant_in_Findlay_BRCA1_Ring_Function_Scores
&include=Variant_in_GnomAD
&include=Variant_in_GnomADv3
"""
    response = rq.get(query_brca2.replace('\n', '')).json()['data']
    with open('brca2_exchange', 'w') as brca2_data:
        json.dump(response, brca2_data)


In [462]:
#2
#BRCA exchange

# https://github.com/BRCAChallenge/brca-exchange/blob/master/website/content/api_docs/api_overview.md

### Generating a list of numbers and converting them from integers into strings. The script will use this list as a comparison, if the search output gives a nucleotide sequence of +5 or -5 it will not be given as output because the list of disallowed numbers ranges from 4 to 9999 in this case, this can be changed below.
pattern = list(range(4, 9999))
pattern_string = [str(number) for number in pattern]

document.add_heading("BRCA Exchange")
table_word = document.add_table(rows=1, cols=4)
table_word.style = 'Colorful List Accent 1'
hdr_cells = table_word.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'HGVS Nucleotide'
hdr_cells[2].text = 'HGVS Protein'
hdr_cells[3].text = 'Clinical Classification'

search = "c.8954-5A>G"
### The script will split and transform the search input

if brca_input == "BRCA1":
    full_path_brca = new_path + brca1_exchange

    f = open (full_path_brca, "r")

    brca_data = json.loads(f.read())

    if ("+" or "-") in search:
        # if + or - is specified in the search the script will NOT split the search string and compare it to the list.
        for data in brca_data:
            if search in data["HGVS_cDNA"]:
                row_cells = table_word.add_row().cells
                row_cells[0].text = brca_input
                cDNA = data["HGVS_cDNA"]
                # The accession number of each result is attached to the output in this case, it will first be splitted of.
                cDNA_split = cDNA.split(":")
                row_cells[1].text = cDNA_split[1]
                row_cells[2].text = data["HGVS_Protein"]
                row_cells[3].text = data["Pathogenicity_expert"]
                
    for data in brca_data:
        if search in data["HGVS_cDNA"]:
            #It will print the outcome if there is no + in the output
            if ("+" or "-") not in data["HGVS_cDNA"]:
                row_cells = table_word.add_row().cells
                # Removing the accession numbers before the nucleotide/protein
                row_cells[0].text = brca_input
                cDNA = data["HGVS_cDNA"]
                cDNA_split = cDNA.split(":")
                row_cells[1].text = cDNA_split[1]

                pDNA = data["HGVS_Protein"]
                pDNA_split = pDNA.split(":")
                row_cells[2].text = pDNA_split[1]                
                row_cells[3].text = data["Pathogenicity_expert"]

            #If there is a plus sign in the output it will start to deconstruct the string and search the for the nucleotide numbers in the range. If there is not a match it will print the output.
            elif "+" in data["HGVS_cDNA"]:

                hit = data["HGVS_cDNA"]

                # In each step the string will be further deconstructed resulting in only the nucleotide sequence
                plus = hit.split("+")
                plus_two = plus[1]
                plus_three = re.split('([a-zA-Z\s]+)', plus_two)
                plus_four = plus_three[0]
                plus_five = plus_four.split('_')
                plus_six = plus_five[0]


                if not any(i == plus_six for i in pattern_string):
                    row_cells = table_word.add_row().cells
                    row_cells[0].text = brca_input
                    cDNA = data["HGVS_cDNA"]
                    cDNA_split = cDNA.split(":")
                    row_cells[1].text = cDNA_split[1]
                   

                    pDNA = data["HGVS_Protein"]
                    pDNA_split = pDNA.split(":")
                    row_cells[2].text = pDNA_split[1]                
                    row_cells[3].text = data["Pathogenicity_expert"]


            elif "-" in data["HGVS_cDNA"]:

                minus = hit.split("-")
                minus_two = minus[1]
                minus_three = re.split('([a-zA-Z\s]+)', plus_minus)
                minus_four = minus_three[0]
                minus_five = minus_four.split('_')
                minus_six = minus_five[0]

                if not any( i == minus_six for i in pattern_string):
                    row_cells = table_word.add_row().cells
                    row_cells[0].text = brca_input
                    cDNA = data["HGVS_cDNA"]
                    cDNA_split = cDNA.split(":")
                    row_cells[1].text = cDNA_split[1]

                    pDNA = data["HGVS_Protein"]
                    pDNA_split = pDNA.split(":")
                    row_cells[2].text = pDNA_split[1]                
                    row_cells[3].text = data["Pathogenicity_expert"]


    #### if there is a match in the protein key it will also extract the data as output

        elif search in data["HGVS_Protein"]:
                row_cells = table_word.add_row().cells
                row_cells[0].text = brca_input
                cDNA = data["HGVS_cDNA"]
                cDNA_split = cDNA.split(":")
                row_cells[1].text = cDNA_split[1]

                pDNA = data["HGVS_Protein"]
                pDNA_split = pDNA.split(":")
                row_cells[2].text = pDNA_split[1]                
                row_cells[3].text = data["Pathogenicity_expert"]

if brca_input == "BRCA2":
    full_path_brca = new_path + brca2_exchange

    f = open (full_path_brca, "r")

    brca_data = json.loads(f.read())

    if ("-") in search:
        for data in brca_data:
            if search in data["HGVS_cDNA"]:
                row_cells = table_word.add_row().cells
                row_cells[0].text = brca_input
                cDNA = data["HGVS_cDNA"]
                cDNA_split = cDNA.split(":")
                row_cells[1].text = cDNA_split[1]
                row_cells[2].text = data["HGVS_Protein"]
                row_cells[3].text = data["Pathogenicity_expert"]
                print(search)

    for data in brca_data:
        if search in data["HGVS_cDNA"]:
            #It will print the outcome if there is no + in the output
            if ("+" or "-") not in data["HGVS_cDNA"]:
                row_cells = table_word.add_row().cells
                # Removing the accession numbers before the nucleotide/protein
                row_cells[0].text = brca_input
                cDNA = data["HGVS_cDNA"]
                cDNA_split = cDNA.split(":")
                row_cells[1].text = cDNA_split[1]

                pDNA = data["HGVS_Protein"]
                pDNA_split = pDNA.split(":")
                row_cells[2].text = pDNA_split[1]                
                row_cells[3].text = data["Pathogenicity_expert"]

            #If there is a plus sign in the output it will start to deconstruct the string and search the for the nucleotide numbers in the range. If there is not a match it will print the output.
            elif "+" in data["HGVS_cDNA"]:

                hit = data["HGVS_cDNA"]

                # In each step the string will be further deconstructed resulting in only the nucleotide sequence
                plus = hit.split("+")
                plus_two = plus[1]
                plus_three = re.split('([a-zA-Z\s]+)', plus_two)
                plus_four = plus_three[0]
                plus_five = plus_four.split('_')
                plus_six = plus_five[0]


                if not any(i == plus_six for i in pattern_string):
                    row_cells = table_word.add_row().cells
                    row_cells[0].text = brca_input
                    cDNA = data["HGVS_cDNA"]
                    cDNA_split = cDNA.split(":")
                    row_cells[1].text = cDNA_split[1]

                    pDNA = data["HGVS_Protein"]
                    pDNA_split = pDNA.split(":")
                    row_cells[2].text = pDNA_split[1]                
                    row_cells[3].text = data["Pathogenicity_expert"]


            elif "-" in data["HGVS_cDNA"]:

                minus = hit.split("-")
                minus_two = minus[1]
                minus_three = re.split('([a-zA-Z\s]+)', plus_minus)
                minus_four = minus_three[0]
                minus_five = minus_four.split('_')
                minus_six = minus_five[0]

                if not any( i == minus_six for i in pattern_string):
                    row_cells = table_word.add_row().cells
                    row_cells[0].text = brca_input
                    cDNA = data["HGVS_cDNA"]
                    cDNA_split = cDNA.split(":")
                    row_cells[1].text = cDNA_split[1]

                    pDNA = data["HGVS_Protein"]
                    pDNA_split = pDNA.split(":")
                    row_cells[2].text = pDNA_split[1]                
                    row_cells[3].text = data["Pathogenicity_expert"]


    #### if there is a match in the protein key it will also extract the data as output

        elif search in data["HGVS_Protein"]:
                row_cells = table_word.add_row().cells
                row_cells[0].text = brca_input
                cDNA = data["HGVS_cDNA"]
                cDNA_split = cDNA.split(":")
                row_cells[1].text = cDNA_split[1]

                pDNA = data["HGVS_Protein"]
                pDNA_split = pDNA.split(":")
                row_cells[2].text = pDNA_split[1]                
                row_cells[3].text = data["Pathogenicity_expert"]

#create and save a hyperlink to word
current_url = "https://brcaexchange.org/variants?search={}%20{}".format(brca_input, search)
p = document.add_paragraph("Results: ")    
site_location = str(current_url)
add_hyperlink(p, 'Link', current_url)


<Element {http://schemas.openxmlformats.org/wordprocessingml/2006/main}hyperlink at 0x248c23728c0>

In [437]:
# 3
# Searching in the LOVD database using selenium webscraping.
# The path to the chromedriver needs to be defined in PATH
# driver will open and interact with the website and has an optional option
# In this case the option is 'headless' which means the driver won't visibly open the website and do the search
# removing the options=option part will disable this so you can follow how the search is done

# The elements can be found by rightclicking on the page when visiting the website and clicking inspect
# The script finds the elements mostly by xpath
pattern = list(range(4, 9999))
pattern_string = [str(number) for number in pattern]
option = webdriver.ChromeOptions()

option.add_argument('headless')
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=option)

#Try is used so the script doesn't fail completely if something goes wrong with one website (it doesn't load, the site got updated, it can't find the search, ...)
try:

    if brca_input == "BRCA1":

        # The script will search for the searchbar on the site and enter the nucleotide search in it.

        driver.get("https://databases.lovd.nl/shared/variants/BRCA1/unique")
        searchbar = driver.find_element_by_name("search_VariantOnTranscript/DNA")
        searchbar.send_keys(search)
        searchbar.send_keys(Keys.RETURN)
    
        #time.sleep() is used to pause the script to give the website the time to load

        time.sleep(1)

        document.add_heading("LOVD")
        table_word = document.add_table(rows=1, cols=4)
        table_word.style = 'Colorful List Accent 1'
        hdr_cells = table_word.rows[0].cells
        hdr_cells[0].text = 'Gene'
        hdr_cells[1].text = 'DNA Change (cDNA)'
        hdr_cells[2].text = 'Clinical Classification'
        hdr_cells[3].text = 'Protein'

        hit = False
    # The script will search for the results table using xpath and iterate over it. hit = False is used so it only adds the relevant rows containing the search query.

        count = 0
        table = driver.find_element_by_xpath("//table[@id='viewlistTable_CustomVL_VOTunique_VOG_BRCA1']")
        for row in table.find_elements_by_xpath("//tr[@class='data']"):
            #The script will iterate over the results table in the page row by row
            count = 0
            for table_data in row.find_elements_by_xpath(".//td"):
                count = count + 1
                # For a row, the script will iterate over each cell
                # It will keep a count for each time it goes over a cell
                # when it reaches the 4th cell AND the search is in this cell it will append the information to the word document
                # hit = True for this row so the script knows to append the information of the 9th cell to the word document aswell
                if count == 4 and search in table_data.text:
                    if ("+" or "-") in search:
                        row_cells = table_word.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[1].text = table_data.text
                        hit = True
            

                    elif ("+" or "-") not in table_data.text:
                        row_cells[0].text = brca_input
                        row_cells[1].text = table_data.text
                        hit = True
                    
                    elif "+" in table_data.text:
                    
                        hit = table_data.text      
                        plus = hit.split("+")
                        plus_two = plus[1]
                        plus_three = re.split('([a-zA-Z\s]+)', plus_two)
                        plus_four = plus_three[0]
                        plus_five = plus_four.split('_')
                        plus_six = plus_five[0]

                        if not any(i == plus_six for i in pattern_string):

                            row_cells = table_word.add_row().cells
                            row_cells[0].text = brca_input
                            row_cells[1].text = table_data.text
                            hit = True

                    elif "-" in table_data.text:

                        minus = hit.split("-")
                        minus_two = minus[1]
                        minus_three = re.split('([a-zA-Z\s]+)', plus_minus)
                        minus_four = minus_three[0]
                        minus_five = minus_four.split('_')
                        minus_six = minus_five[0]

                        if not any( i == minus_six for i in pattern_string):
                            row_cells = table_word.add_row().cells
                            row_cells[0].text = brca_input
                            row_cells[1].text = table_data.text
                            hit = True

                elif count == 6 and search in table_data.text:
                    row_cells[O].text = brca_input
                    row_cells[3].text = table_data.text
                    hit = True


                if count == 9 and hit == True:
                    row_cells[2].text = table_data.text
                    hit = False               



        # It will append the results to the word document alongside with a hyperlink of the current url location
        p = document.add_paragraph("Results: ")    
        site_location = str(driver.current_url)
        add_hyperlink(p, 'Link', driver.current_url)

        driver.quit()

    elif brca_input == "BRCA2":
        #If BRCA2 is given as input it will start the search on the BRCA2 tab of the website
        driver.get("https://databases.lovd.nl/shared/variants/BRCA2/unique")
        searchbar = driver.find_element_by_name("search_VariantOnTranscript/DNA")
        searchbar.send_keys(search)
        searchbar.send_keys(Keys.RETURN)
    
        time.sleep(1)

        document.add_heading("LOVD")
        table_word = document.add_table(rows=1, cols=3)
        table_word.style = 'Colorful List Accent 1'
        hdr_cells = table_word.rows[0].cells
        hdr_cells[0].text = 'Gene'
        hdr_cells[1].text = 'DNA Change (cDNA)'
        hdr_cells[2].text = 'Clinical Classification'
    
        hit = False
    
        count = 0
        table = driver.find_element_by_xpath("//table[@id='viewlistTable_CustomVL_VOTunique_VOG_BRCA2']")
        for row in table.find_elements_by_xpath("//tr[@class='data']"):
            count = 0
            for table_data in row.find_elements_by_xpath(".//td"):
                count = count + 1
                if count == 4 and search in table_data.text:
                    # Searching on nucleotide level
                    if ("+" or "-") in search:
                        row_cells = table_word.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[1].text = table_data.text
                        hit = True

                    elif ("+" or "-") not in table_data.text:
                        row_cells[0].text = brca_input
                        row_cells[1].text = table_data.text
                        hit = True
                    
                    elif "+" in table_data.text:
                    
                        hit = table_data.text      
                        plus = hit.split("+")
                        plus_two = plus[1]
                        plus_three = re.split('([a-zA-Z\s]+)', plus_two)
                        plus_four = plus_three[0]
                        plus_five = plus_four.split('_')
                        plus_six = plus_five[0]

                        if not any(i == plus_six for i in pattern_string):

                            row_cells = table_word.add_row().cells
                            row_cells[0].text = brca_input
                            row_cells[1].text = table_data.text
                            hit = True

                    elif "-" in table_data.text:

                        minus = hit.split("-")
                        minus_two = minus[1]
                        minus_three = re.split('([a-zA-Z\s]+)', plus_minus)
                        minus_four = minus_three[0]
                        minus_five = minus_four.split('_')
                        minus_six = minus_five[0]

                        if not any( i == minus_six for i in pattern_string):
                            row_cells = table_word.add_row().cells
                            row_cells[0].text = brca_input
                            row_cells[1].text = table_data.text
                            hit = True

                elif count == 6 and search in table_data.text:
                    # Searching on protein lev
                    row_cells[O].text = brca_input
                    row_cells[3].text = table_data.text
                    hit = True

                if count == 9 and hit == True:
                    row_cells[2].text = table_data.text
                    hit = False

        p = document.add_paragraph("Results: ")    
        site_location = str(driver.current_url)
        add_hyperlink(p, 'Link', driver.current_url)
        print(driver.current_url)

        driver.quit()
    document.save("test.docx")

except Exception:
    pass

https://databases.lovd.nl/shared/variants/BRCA2/unique#object_id=VariantOnTranscriptUnique%2CVariantOnGenome&id=BRCA2&order=VariantOnTranscript%2FDNA%2CASC&search_transcriptid=00003479&search_VariantOnTranscript/DNA=c.8954-5A%3EG&page_size=100&page=1


In [438]:

# Lindor pdf
# 4
# The script will read the pdf file containing the tables
# It will create a folder and paste the tables in excel files, this step is skipped if this folder already exists
# The script will iterate over each excel file in the folder


tables_pdf = read_pdf(folder_location + pdf_lindor, pages="17-28")
tables_output_location = "/tables excel lindor 2012"

tables_folder = folder_location + tables_output_location

if not os.path.exists(tables_folder):
    os.makedirs(tables_folder)

for i, table_pdf in enumerate(tables_pdf, start=1):
    table_pdf.to_excel(os.path.join(tables_folder, f"table_{i}.xlsx"), index=False)

# The excel files are divided in different groups based on the similarity of their structure (Equal amount of columns, columns that have the same meaning)
group_one = ('table_1.xlsx')
group_two = ("2.xlsx", "3.xlsx", "4.xlsx", "5.xlsx")
group_three = ("6.xlsx", "7.xlsx", "8.xlsx", "9.xlsx", "10.xlsx")
group_four = ("11.xlsx")
group_five = ("12.xlsx")

document.add_heading("Lindor 2012 pdf")

table = document.add_table(rows=1, cols=5)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'HGSV nucleotide'
hdr_cells[2].text = 'BIC nucleotide'
hdr_cells[3].text = "HGVS protein"
hdr_cells[4].text = 'Class'

tables_dir = os.listdir(tables_folder)

#In the first half it will search for BRCA1 genes

for table_file in tables_dir:
    if brca_input == "BRCA1":
        if table_file.endswith(group_one):

            wb = load_workbook(tables_folder + "/" + table_file)
            sheet = wb.active

            for row in sheet.iter_rows():
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[3].text = brca_input
                        row_cells[4].text = str(cell.offset(column=5).value)
                        #protein?


        if table_file.endswith(group_two):

            wb = load_workbook(tables_folder + "/" + table_file)
            sheet = wb.active

            for row in sheet.iter_rows(max_col = 3, min_col = 3):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[2].text = cell.value
                        row_cells[3].text = str(cell.offset(column=-1).value)
                        row_cells[4].text = str(cell.offset(column=5).value)
                        #bic
                        
            for row in sheet.iter_rows(max_col = 4, min_col = 4):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[1].text = cell.value
                        row_cells[2].text = str(cell.offset(column=-2).value)
                        row_cells[4].text = str(cell.offset(column=4).value)
                        #hgvs

            for row in sheet.iter_rows(max_col = 2, min_col = 2):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[1].text = str(cell.offset(column=2).value)
                        row_cells[2].text = str(cell.offset(column=1).value)
                        row_cells[3].text = cell.value
                        row_cells[4].text = str(cell.offset(column=6).value)
                        #hgvs protein
                        
            
        if table_file.endswith(group_four):

            wb = load_workbook(tables_folder + "/" + table_file)
            sheet = wb.active
            for row in sheet.iter_rows():
                for cell in row:
                    if brca_input and search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = cell.value
                        row_cells[4].text = str(cell.offset(column=2).value)
                        #brca1/brca2
#Second half: it will search for the BRCA2 gene in the applicable file

    if brca_input == "BRCA2":
        if table_file.endswith(group_three):

            wb = load_workbook(tables_folder + "/" + table_file)
            sheet = wb.active
            for row in sheet.iter_rows(max_col = 3, min_col = 3):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[2].text = cell.value
                        row_cells[3].text = str(cell.offset(column=-1).value)
                        row_cells[4].text = str(cell.offset(column=8).value)
                        #bic

            for row in sheet.iter_rows(max_col = 4, min_col = 4):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[1].text = cell.value
                        row_cells[3].text = str(cell.offset(column=-2).value)
                        row_cells[4].text = str(cell.offset(column=5).value)
                        #hgvs

            for row in sheet.iter_rows(max_col = 2, min_col = 2):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[1].text = str(cell.offset(column=2).value)
                        row_cells[2].text = str(cell.offset(column=1).value)
                        row_cells[3].text = cell.value
                        row_cells[4].text = str(cell.offset(column=9).value)
                        #hgvs protein


        if table_file.endswith(group_four):

            wb = load_workbook(tables_folder + "/" + table_file)
            sheet = wb.active
            for row in sheet.iter_rows():
                for cell in row:
                    if brca_input and search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = cell.value
                        row_cells[3].text = str(cell.offset(column=2).value)
                        
        if table_file.endswith(group_five):

            wb = load_workbook(tables_folder + "/" + table_file)
            sheet = wb.active
            for row in sheet.iter_rows(max_col = 4, min_col = 4):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[1].text = cell.value
                        row_cells[3].text = str(cell.offset(column=5).value)
                        #hgvs

                     
            for row in sheet.iter_rows(max_col = 5, min_col = 5):
                for cell in row:
                    if search in str(cell.value):
                        row_cells = table.add_row().cells
                        row_cells[0].text = brca_input
                        row_cells[2].text = cell.value
                        row_cells[3].text = str(cell.offset(column=4).value)
                        #bic


In [439]:
#5) Reading excel sheet: bicbnl 27  fh 2-10-13_nov2013
# Python gives warning it can't parse the header but the script still works
# Similar to the other excel scripts except it differentiates between two tabs first.

wb = load_workbook(folder_location + bicbnl)

document.add_heading("LOB")

table = document.add_table(rows=1, cols=5)
table.allow_outfit = True

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'c.nom'
hdr_cells[2].text = 'Type'
hdr_cells[3].text = 'opm LOB'
hdr_cells[4].text = 'opmerking'
    
if brca_input == "BRCA1":
    # The script will open the right sheet first.
    sheet = wb['totaal BRCA1']

    for row in sheet.iter_rows(max_col = 5, min_col = 5):
        # It will search in the 5th column for the nucleotide
        for cell in row:
            if search in str(cell.value):
                row_cells = table.add_row().cells
                row_cells[0].text = brca_input
                row_cells[1].text = cell.value
                row_cells[2].text = str(cell.offset(column=2).value) # column G (type)
                row_cells[3].text = str(cell.offset(column=14).value) # column S (opmerking LOB indeling)
                row_cells[4].text = str(cell.offset(column=15).value) # column T (opmerking) 

elif brca_input == "BRCA2":
    sheet = wb['totaal BRCA2']

    for row in sheet.iter_rows(max_col = 5, min_col = 5):
        for cell in row:
            if search in str(cell.value):
                row_cells = table.add_row().cells
                row_cells[0].text = brca_input
                row_cells[1].text = cell.value
                row_cells[2].text = str(cell.offset(column=2).value)
                row_cells[3].text = str(cell.offset(column=14).value)
                row_cells[4].text = str(cell.offset(column=15).value)



C:\Users\jensv\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [440]:
from eutils import Client
from Bio import Entrez
import requests as rq
import pprint
#4/5/6 clinvar
eclient = Client(api_key="84c4d9fa268d5e41511ce024a0cf00537808")

gene_esearch = eclient.esearch(db='clinvar',term='{}[gene] AND {}'.format(brca_input, search))

id_list = gene_esearch.ids

document.add_heading("Clinvar")
table_word = document.add_table(rows=1, cols=4)
table_word.style = 'Colorful List Accent 1'
hdr_cells = table_word.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'cDNA'
hdr_cells[2].text = 'Classification'
hdr_cells[3].text = 'Review Status'


for i in id_list:
    query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={}&retmode=json".format(i)
    response = rq.get(query).json()
    id_string = str(i)
    if search in response["result"][id_string]["title"]:
        row_cells = table_word.add_row().cells
        row_cells[0].text = brca_input
        sequence = response["result"][id_string]["title"]
        sequence_split = sequence.split(":")
        row_cells[1].text = sequence_split[1]
        row_cells[2].text = response["result"][id_string]["clinical_significance"]["description"]
        if response["result"][id_string]["clinical_significance"]["review_status"] == "practice guideline":
            row_cells[3].text = "****"
        elif response["result"][id_string]["clinical_significance"]["review_status"] == "reviewed by expert panel":
            row_cells[3].text = "***"
        elif response["result"][id_string]["clinical_significance"]["review_status"] == "criteria provided, multiple submitters, no conflicts":
            row_cells[3].text = "**"
        elif response["result"][id_string]["clinical_significance"]["review_status"] == "criteria provided, conflicting interpretations" or 'criteria provided, single submitter':
            row_cells[3].text = "*"
        elif response["result"][id_string]["clinical_significance"]["review_status"] == "no assertion for the individual variant" or "no assertion criteria provided" or "no assertion provided":
            row_cells[3].text = "/"

current_url = "https://www.ncbi.nlm.nih.gov/clinvar/?term={}%5Bgene%5D+{}".format(brca_input, search)
p = document.add_paragraph("Results: ")    
site_location = current_url
add_hyperlink(p, 'Link', current_url)

<Element {http://schemas.openxmlformats.org/wordprocessingml/2006/main}hyperlink at 0x248c178db40>

In [441]:
"""
import re
test = 'c.301+7G>A'
test_split = re.split("[+, ., -, >, d-zD-Z]", test)
print(test_split)
"""


'\nimport re\ntest = \'c.301+7G>A\'\ntest_split = re.split("[+, ., -, >, d-zD-Z]", test)\nprint(test_split)\n'

In [442]:
#Enigma rules manual made table
# 4?
enigma_table_brca1 = "/ENIGMA RULES BRCA1.xlsx"
enigma_table_brca2 = "/ENIGMA RULES BRCA2.xlsx"


search_split = re.split("[\-!?:.a-zA-Z]+", search)
search_split = search_split[1]
int_search = int(search_split)

'''
if (".") in search:
    search_split = re.split("[.]", search)
    search_split = search_split[1]

if ("+" or "-" ) in search:
    search_split = re.split("[+, ., -, >, d-zD-Z]", search_split)
    int_search = int(search_split[0])
elif (">" or "del") in search:
    search_split = re.split("[+, ., -, >, d-zD-Z]", search_split)
    int_search = int(search_split[0])
    print(search_split[1])
if ("del") in search:
    search_split = re.split("[d-zD-Z]", search_split)
else:
    int_search = int(search)
'''
if brca_input == "BRCA1":
    wb = load_workbook(folder_location + enigma_table_brca1)

    sheet = wb.active

    document.add_heading("Table 3: Catalogue of BRCA1 conserved domains/motifs and currently known clinically important amino acid residues, and relevance for classification of BRCA1 in-frame and terminal exon sequence variants.")

    table = document.add_table(rows=1, cols=5)

    table.style = 'Colorful List Accent 1'
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Domain/motif'
    hdr_cells[1].text = 'AA start'
    hdr_cells[2].text = 'AA end'
    hdr_cells[3].text = 'AA alterations'
    hdr_cells[3].text = 'Classification'

    # searching for between AA start and AA end
    for row in sheet.iter_rows(min_row=2, min_col=2, max_col=2):
        for cell in row:
            if int_search >= int(cell.value) and int_search <= int(cell.offset(column=1).value):
                row_cells = table.add_row().cells
                row_cells[0].text = brca_input
                AA_start = str(cell.value)
                AA_end = str(cell.offset(column=1).value)
                classification = str(cell.offset(column=3).value)

            #  print("{} is found between {} and {} and has classification {}".format(search, AA_start, AA_end, classification))
            
    ###AA alteration with demonstrated clinical importance
    for row in sheet.iter_rows(min_col=4, max_col=4):
        for cell in row:
            if search in str(cell.value):
                row_cells = table.add_row().cells
                AA_alteration = cell.value ####grep?
                classification = str(cell.offset(column=1).value)

if brca_input == "BRCA2":  

    wb = load_workbook(folder_location + enigma_table_brca2)

    sheet = wb.active

    document.add_heading("Table 3: Catalogue of BRCA1 conserved domains/motifs and currently known clinically important amino acid residues, and relevance for classification of BRCA1 in-frame and terminal exon sequence variants.")
    #row_cells[2].text = str(cell.offset(column=2).value)

    ###BRCA2

    table = document.add_table(rows=1, cols=5)

    table.style = 'Colorful List Accent 1'
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Domain/motif'
    hdr_cells[1].text = 'AA start'
    hdr_cells[2].text = 'AA end'
    hdr_cells[3].text = 'AA alterations'
    hdr_cells[3].text = 'Classification'

    # searching for between AA start and AA end
    for row in sheet.iter_rows(min_row=2, min_col=2, max_col=2):
        for cell in row:
            if int_search >= int(cell.value) and int_search <= int(cell.offset(column=1).value):
                row_cells = table.add_row().cells
                row_cells[0].text = brca_input
                AA_start = str(cell.value)
                AA_end = str(cell.offset(column=1).value)
                classification = str(cell.offset(column=3).value)

                #print("{} is found between {} and {} and has classification {}".format(search, AA_start, AA_end, classification))
            
    ###AA alteration with demonstrated clinical importance
    for row in sheet.iter_rows(min_col=4, max_col=4):
        for cell in row:
            if search in str(cell.value):
                row_cells = table.add_row().cells
                AA_alteration = cell.value ####grep?
                classification = str(cell.offset(column=1).value)





In [443]:
#2) Reading excel sheet : allenigmavariants_BICsubmission_2013-07-01
#7A

# The first column in this file is BRCA1/BRCA2, the second column is hgvsnucleotide and the fourth column is bic nucleotide
# The script will iterate over each row, if the brca_input matches with the gene in the 1st column AND the search is found in the 2nd OR 4th column it will give the class in column 7 as output

#cell.offset(column=2).value gives an error as output if the cell is empty but not when it is converted to string first.

# Zoeken op protein change
wb = load_workbook(folder_location + variants_excel)

sheet = wb.active

document.add_heading("ENIGMA class 1 lijst met SNVs (freq >1>%% in 1000g)")

table = document.add_table(rows=1, cols=4)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'HGSV nucleotide'
hdr_cells[2].text = 'HGSV protein'
hdr_cells[3].text = 'Class'


for row in sheet.iter_rows():
    for cell in row:
        # if BRCA gene is in 1st column AND nucleotide search is in 2ndcolumn (hgvs nucleotide)
        if brca_input in str(cell.value) and search in str(cell.offset(column=1).value) or brca_input in str(cell.value) and search in str(cell.offset(column=2).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value #gene
            row_cells[1].text = cell.offset(column=1).value #hgvs nucleotide
            row_cells[2].text = str(cell.offset(column=2).value) #hgvs protein
            row_cells[3].text = str(cell.offset(column=6).value) #class
        '''
        #else if BRCA gene is in 1st column AND nucleotide search is in 4th column (bic nucleotide)  
        elif brca_input in str(cell.value) and search in str(cell.offset(column=3).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value #gene
            row_cells[2].text = cell.offset(column=3).value #bic nucleotide
            row_cells[3].text = cell.offset(column=6).value #class
        '''



In [444]:
#3)Reading excel sheet: Published Multifactoral data enigma
#7B samen met 7A
# This is likewise to searching in the allenigmavariants_BICsubmission_2013-07-01 excel file


wb = load_workbook(folder_location + enigma)

sheet = wb.active

document.add_heading("ENIGMA MLM lijst ( variants classified based on multifactorial likelihood model)")

table = document.add_table(rows=1, cols=4)

table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'HGSV nucleotide'
hdr_cells[2].text = 'HGVS protein'
hdr_cells[3].text = 'Class'


for row in sheet.iter_rows():
    for cell in row:
        if brca_input in str(cell.value) and search in str(cell.offset(column=1).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value
            row_cells[1].text = cell.offset(column=1).value
            row_cells[2].text = str(cell.offset(column=2).value)
            row_cells[3].text = cell.offset(column=6).value
            Result = False
        if brca_input in str(cell.value) and search in str(cell.offset(column=2).value):
            row_cells[0].text = cell.value
            row_cells[1].text = cell.offset(column=1).value
            row_cells[2].text = str(cell.offset(column=2).value)
            row_cells[3].text = cell.offset(column=6).value

            '''
        elif brca_input in str(cell.value) and search in str(cell.offset(column=3).value):
            row_cells = table.add_row().cells
            row_cells[0].text = cell.value
            row_cells[2].text = cell.offset(column=3).value
            row_cells[3].text = cell.offset(column=6).value
            Result = False
            '''


In [445]:
#1) Reading excel sheet: 2020 functional categorization of BRCA1 VUS - CCR
# Zoeken op protein
# 8
# Bij group B SNVS smaen met enigma en function scores

### Add heading to word document
document.add_heading("BRCA1 SNV lijst met functionele data (PMID: 32546644)", 1)

### Create table in word
table = document.add_table(rows=1, cols=4)

### Loading a style template and creating the header cells
table.style = 'Colorful List Accent 1'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'DNA variant'
hdr_cells[1].text = 'Cisplatin Assay'
hdr_cells[2].text = 'Olaparib Assay'
hdr_cells[3].text = 'DR-GFP assay'


#If the input is BRCA1 the script will load the excel file. It will iterate over each row of the first column (max_col = 1) of the active sheet.
#If the search input is found in the first column the script will add the values of the 3rd, 4th and 5th column
#Example cell.offset(column=2).value means that the script will take the value of the 3rd column (1st column + 2 to the right = 3)
if brca_input == "BRCA1":

    wb = load_workbook(folder_location + functional_categorization)

    sheet = wb.active

    for row in sheet.iter_rows(max_col = 2, min_col = 2):
        for cell in row:
            if search in str(cell.value):
                    row_cells = table.add_row().cells
                    row_cells[0].text = cell.value
                    row_cells[1].text = cell.offset(column=1).value
                    row_cells[2].text = cell.offset(column=2).value
                    row_cells[3].text = cell.offset(column=3).value


In [446]:
########Franklin database
#9
try:

    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    driver = webdriver.Chrome(PATH, options=option)

    document.add_heading("Franklin")
    table_word = document.add_table(rows=1, cols=2)
    table_word.style = 'Colorful List Accent 1'
    hdr_cells = table_word.rows[0].cells
    hdr_cells[0].text = 'Gene'
    hdr_cells[1].text = 'Classification'
    row_cells = table_word.add_row().cells

    driver.get("https://franklin.genoox.com/clinical-db/home")

    time.sleep(10)

    #a pop up appeared on this page which can be removed by pressing escape
    ActionChains(driver).send_keys(Keys.ESCAPE).perform()

    #The script will search for the searchbar and the button(somatic)
    searchbar = driver.find_element_by_xpath("/html/body/app-root/div/gnx-home-page/div/gnx-search/div      [2]/input")

    button = driver.find_element_by_xpath("/html/body/app-root/div/gnx-home-page/div/gnx-search/div[2]/     gnx-variant-type-toggle/button[2]")

    button.click()

    searchbar.send_keys(search)
    searchbar.send_keys(Keys.RETURN)

    #time.sleep(10)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "mat-tab-label-2-3")))

    except TimeoutException:
        print("Timed out waiting for page to load")

    driver.find_element_by_xpath("//*[@id='mat-tab-label-2-3']").click()

    time.sleep(3)

    classification = driver.find_element_by_xpath("//*[@class='indicator-text ng-star-inserted']")

    row_cells[0].text = brca_input
    row_cells[1].text = classification.text
    franklin_class = classification.text


    p = document.add_paragraph("Results: ")    
    site_location = str(driver.current_url)
    add_hyperlink(p, 'Link', driver.current_url)

    driver.quit()

except Exception:
    pass


In [447]:
#Varsome login required if too many searches are done
#10
try:

    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    driver = webdriver.Chrome(PATH, options=option)

    driver.get("https://varsome.com/")

    document.add_heading("Varsome")
    table_word = document.add_table(rows=1, cols=2)
    table_word.style = 'Colorful List Accent 1'
    hdr_cells = table_word.rows[0].cells
    hdr_cells[0].text = "Gene"
    hdr_cells[1].text = 'Classification'
    row_cells = table_word.add_row().cells

    time.sleep(1)

    searchbar = driver.find_element_by_xpath("//*[@id='search']")
    searchbar.send_keys(brca_input + " " + search)
    searchbar.send_keys(Keys.RETURN)

    time.sleep(1)

    #Popup asking to accept cookies 
    cookie = driver.find_element_by_xpath("//*[@id='onetrust-accept-btn-handler']")
    cookie.click()

    button = driver.find_element_by_xpath("//*[@id='proceedBtn']/h4")
    button.click()

    time.sleep(10)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "acmg_annotation")))

    except TimeoutException:
        print("Timed out waiting for page to load")
    

    classification = driver.find_element_by_xpath("//*[@id='acmg_annotation']/div/div[1]/div/div[2]/div/    div/div[1]/div/div/span/div[2]/div/div")

    row_cells = table_word.add_row().cells
    row_cells[0].text = brca_input + " " + search
    row_cells[1].text = classification.text

    p = document.add_paragraph("Results: ")    
    site_location = str(driver.current_url)
    add_hyperlink(p, 'Link', driver.current_url)
    print(driver.current_url)

    driver.quit()

except Exception:
    pass


https://varsome.com/variant/hg19/BRCA2%20c.8954-5A%3EG?annotation-mode=germline


In [448]:
#Database of Functional Classifications of BRCA1 Variants based on Saturation Genome Editing
#
try:
    
    if brca_input == "BRCA1":

        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        PATH = "C:\Program Files (x86)\chromedriver.exe"
        driver = webdriver.Chrome(PATH, options=option)

        document.add_heading("Database of Functional Classifications of BRCA1 Variants based on Saturation Genome Editing")
        table_word = document.add_table(rows=1, cols=3)
        table_word.style = 'Colorful List Accent 1'
        hdr_cells = table_word.rows[0].cells
        hdr_cells[0].text = 'Gene'
        hdr_cells[1].text = 'Transcript Variant'
        hdr_cells[2].text = 'Functional CLass'


        driver.get("https://sge.gs.washington.edu/BRCA1/#tab-9159-2")

        time.sleep(1)
        #The script will look up and click on the right tab
        tab = driver.find_element_by_xpath("//*[@id='main_page']/div/div/nav/div/ul/li[2]/a")
        tab.click()
        '''
        tab = driver.find_element_by_xpath("//ul[@class='nav navbar-nav']")
        tab.find_element_by_xpath(".//a[@href='#tab-9159-2']").click()
        '''
        #time.sleep(5)
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "DataTables_Table_0_filter")))

        except TimeoutException:
            print("Timed out waiting for page to load")

        # look up the searchbar and enter the query
        searchbar = driver.find_element_by_xpath("//*[@id='DataTables_Table_0_filter']/label/input")
        searchbar.send_keys(search)
        searchbar.send_keys(Keys.RETURN)
    
        table = driver.find_element_by_xpath("//*[@id='DataTables_Table_0']/tbody")
 
        count = 0
        time.sleep(5)
        hit = False

        for row in table.find_elements_by_xpath("//tr"):
            count = 0
            for table_data in row.find_elements_by_xpath((".//td")):
                count = count + 1
                if count == 6 and search in table_data.text:
                    row_cells = table_word.add_row().cells
                    row_cells[0].text = brca_input
                    hit = True
                    row_cells[1].text = table_data.text 
                if count == 10 and hit == True:
                    row_cells[2].text = table_data.text
                    hit = False

    p = document.add_paragraph("Results: ")    
    site_location = str(driver.current_url)
    add_hyperlink(p, 'Link', driver.current_url)
    print(driver.current_url)

    driver.quit()
except Exception:
    pass

In [449]:
#Oncokb.org

try:

    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    driver = webdriver.Chrome(PATH, options = option)

    driver.get("https://www.oncokb.org/")

    document.add_heading("OncoKB")
    table_word = document.add_table(rows=1, cols=2)
    table_word.style = 'Colorful List Accent 1'
    hdr_cells = table_word.rows[0].cells
    hdr_cells[0].text = 'Search'
    hdr_cells[1].text = 'Results'

    #time.sleep(5)

    searchbar = driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[3]/div/div/div[3]/div/div/       div/div[1]/div[2]/div/input")
    searchbar.send_keys(brca_input + " " + search)
    time.sleep(2)
    searchbar.send_keys(Keys.RETURN)
    time.sleep(5)
    result = driver.find_element_by_xpath("//*[@id='root']/div[2]/div[3]/div/div/div/div[2]/div/div[2]")

    row_cells = table_word.add_row().cells
    row_cells[0].text = brca_input + search
    row_cells[1].text = result.text

    p = document.add_paragraph("Results: ")    
    site_location = str(driver.current_url)
    add_hyperlink(p, 'Link', driver.current_url)
    print(driver.current_url)

    driver.quit()
    
except Exception:
    pass

https://www.oncokb.org/gene/BRCA2/c.8954-5A%3EG


In [450]:
# Pubmed
p = document.add_paragraph("pubmed: ") 
pubmed_site = "https://pubmed.ncbi.nlm.nih.gov/?term="  
pubmed_full = pubmed_site + brca_input + "+" + search
add_hyperlink(p, 'Link', pubmed_full)
print(pubmed_full)



https://pubmed.ncbi.nlm.nih.gov/?term=BRCA2+c.8954-5A>G


In [451]:
document.save("output.docx")

In [452]:
'''
pdf_enigma = "/ENIGMA_Rules_2017-06-29.pdf"
import tabula
import tabulate
folder_location = "C:/Users/jensv/Desktop/BRCA_-_prostate_cancer_and_ovarian_cancer_-_PARPi"
tables_enigma = read_pdf(folder_location + pdf_enigma, pages='14-16')
tables_output_location = "/tables ENIGMA"

print(tables_enigma)
tables_folder = folder_location + tables_output_location

if not os.path.exists(tables_folder):
    os.makedirs(tables_folder)

for i, table_pdf in enumerate(tables_enigma, start=1):
    table_pdf.to_excel(os.path.join(tables_folder, f"table_{i}.xlsx"), index=False)
'''

'\npdf_enigma = "/ENIGMA_Rules_2017-06-29.pdf"\nimport tabula\nimport tabulate\nfolder_location = "C:/Users/jensv/Desktop/BRCA_-_prostate_cancer_and_ovarian_cancer_-_PARPi"\ntables_enigma = read_pdf(folder_location + pdf_enigma, pages=\'14-16\')\ntables_output_location = "/tables ENIGMA"\n\nprint(tables_enigma)\ntables_folder = folder_location + tables_output_location\n\nif not os.path.exists(tables_folder):\n    os.makedirs(tables_folder)\n\nfor i, table_pdf in enumerate(tables_enigma, start=1):\n    table_pdf.to_excel(os.path.join(tables_folder, f"table_{i}.xlsx"), index=False)\n'

In [453]:
'''
# API requst varsome 1 request per day
import requests
import json

response = requests.get('https://api.varsome.com/lookup/gene/BRCA1/hg19?add-all-data=1')

print(response.json())

'''


"\n# API requst varsome 1 request per day\nimport requests\nimport json\n\nresponse = requests.get('https://api.varsome.com/lookup/gene/BRCA1/hg19?add-all-data=1')\n\nprint(response.json())\n\n"

In [454]:
'''
#LOVD database
import requests as rq
import json
import pprint
import feedparser
import bs4




query = """
https://databases.lovd.nl/shared/api/rest.php/variants/IVD
"""
response = rq.get("https://databases.lovd.nl/shared/variants/BRCA1/unique1")

responsesoup = bs4.BeautifulSoup(response.text)
for i in responsesoup:
    if "5074" in i:
        print(i)
''''

SyntaxError: EOL while scanning string literal (<ipython-input-454-44987550c2d7>, line 21)

In [23]:
'''
#LOVD database
import requests as rq
import json
import pprint
import feedparser
import bs4
from xml.etree import cElementTree as ET




query = """
https://databases.lovd.nl/shared/api/rest.php/variants/IVD
"""
'''


NameError: name 'BeautifulSoup' is not defined

In [24]:
'''


#token = {{franklin token}}
franklin_token = "franklin token"

query= "https://api.genoox.com/v1/variant/search?search_text=VHL:c.293A>G"

r = rq.get(query, auth='franklin_token')
''''

TypeError: 'str' object is not callable

In [78]:
'''
import myvariant
mv = myvariant.MyVariantInfo()

out = mv.query('clinvar.gene.symbol:BRCA1 AND clinvar.rcv.clinical_significance:pathogenic')
'''